`Done - Initiatives`

Initiatives are actions that firms take to decarbonize, they are highly informative of future behavior. It is interesting to understand which initiatives work best and what is the timeframe of those initiatives. 

- Msaving total and invesmtent total are highly correlated. Msavings is the expected savings, while invesmtent is the investment requried to achieve the initiative.
- Sometimes the initiative investment details are not disclosed. Though there might be explanatory power. Here I can create binary indicators that say whether or not there is an initiative, without necessarily specifying the impact of that initiative. Another potential option could be to create a counter for the number of initatives that a firm has.
- C02 total I believe is the impact of the initiative. Though more research should be done.

Each initiative is divided into c02, msaving and investment.

In [87]:
import pandas as pd
import numpy as np
import seaborn as sns

In [88]:
data = pd.read_stata("../../data/CDP Cleaning/cleaned outputs/cdp_initiative_firm_year.dta")

In [89]:
data.head()

,id,year,co2_behavioral,msaving_behavioral,investment_behavioral,co2_energy_building,msaving_energy_building,investment_energy_building,co2_energy_production,msaving_energy_production,...,msaving_total,investment_total,reporting_boundary,companyname,country,industry14,isin,ticker,industry46,cdpdate
0,44,2011.0,0.0,0.00,0.0,NaN,NaN,NaN,NaN,NaN,...,0.000000,0.000000,,3i Group,United Kingdom,Services,GB00B1YW4409,III LN,banks diverse financials insurance,NaT
1,44,2013.0,NaN,NaN,NaN,NaN,NaN,NaN,901.5,9303.000,...,18606.000000,3101.000000,,3i Group,United Kingdom,Services,GB00B1YW4409,III LN,banks diverse financials insurance,2013-06-18
2,44,2014.0,NaN,NaN,NaN,10.31,3505.33,0.0,NaN,NaN,...,109271.437500,0.000000,,3i Group,United Kingdom,Services,GB00B1YW4409,III LN,banks diverse financials insurance,2014-06-26
3,44,2016.0,4.8,1441.11,1310.1,NaN,NaN,NaN,NaN,NaN,...,1441.109985,1310.099976,,3i Group,United Kingdom,Services,GB00B1YW4409,III LN,banks diverse financials insurance,2016-06-27
4,44,2017.0,NaN,NaN,NaN,NaN,NaN,NaN,2.7,1062.064,...,1062.063965,3188.782471,,3i Group,United Kingdom,Services,GB00B1YW4409,III LN,banks diverse financials insurance,2017-06-20


In [90]:
data.loc[data['id'] == 9759].iloc[2,:]

id                                                     9759
year                                                 2020.0
co2_behavioral                                          NaN
msaving_behavioral                                      NaN
investment_behavioral                                   NaN
co2_energy_building                                     NaN
msaving_energy_building                                 NaN
investment_energy_building                              NaN
co2_energy_production                                   NaN
msaving_energy_production                               NaN
investment_energy_production                            NaN
co2_fugitive                                            NaN
msaving_fugitive                                        NaN
investment_fugitive                                     NaN
co2_low_carbon                                          0.0
msaving_low_carbon                                      0.0
investment_low_carbon                   

In [91]:
data[data.duplicated(subset=['id','year'])]

,id,year,co2_behavioral,msaving_behavioral,investment_behavioral,co2_energy_building,msaving_energy_building,investment_energy_building,co2_energy_production,msaving_energy_production,...,msaving_total,investment_total,reporting_boundary,companyname,country,industry14,isin,ticker,industry46,cdpdate


Here my intuition is that since there are a lot of missing values, the question then becomes wehther they did an initiative or not. In which case I can convert the columns to categorical predictors. Though this will have to be confirmed

In [92]:
# Creating counter columns for 'co2', 'msaving', and 'investment'
# Excluding columns ending with 'total'

# Columns for co2, msaving, and investment
co2_columns = [col for col in data.columns if col.startswith('co2_') and col != 'co2_total']
msaving_columns = [col for col in data.columns if col.startswith('msaving_') and col != 'msaving_total']
investment_columns = [col for col in data.columns if col.startswith('investment_') and col != 'investment_total']

# Counting entries greater than zero for each category
data['co2_counter'] = (data[co2_columns] > 0).sum(axis=1)
data['msaving_counter'] = (data[msaving_columns] > 0).sum(axis=1)
data['investment_counter'] = (data[investment_columns] > 0).sum(axis=1)

# Dropping specific categories columns
data = data.drop(columns=co2_columns + msaving_columns + investment_columns)

# Dropping additional specified columns
columns_to_drop = ['reporting_boundary', 'companyname', 'country', 'industry14', 'industry46', 'ticker', 'cdpdate']
data = data.drop(columns=columns_to_drop)

# Showing the modified dataframe
data.head()


,id,year,co2_total,msaving_total,investment_total,isin,co2_counter,msaving_counter,investment_counter
0,44,2011.0,0.000000,0.000000,0.000000,GB00B1YW4409,0,0,0
1,44,2013.0,1803.000000,18606.000000,3101.000000,GB00B1YW4409,2,2,2
2,44,2014.0,49.980000,109271.437500,0.000000,GB00B1YW4409,2,2,0
3,44,2016.0,4.800000,1441.109985,1310.099976,GB00B1YW4409,1,1,1
4,44,2017.0,602.700012,1062.063965,3188.782471,GB00B1YW4409,2,1,1


I would definitely log1p the variables here!

In [93]:
# Applying log1p transformation to specified columns
data['co2_total_log1p'] = np.log1p(data['co2_total'])
data['msaving_total_log1p'] = np.log1p(data['msaving_total'])
data['investment_total_log1p'] = np.log1p(data['investment_total'])

# Dropping the 'isin' column
data = data.drop(columns=['isin'])

# Showing the modified dataframe
data.head()


,id,year,co2_total,msaving_total,investment_total,co2_counter,msaving_counter,investment_counter,co2_total_log1p,msaving_total_log1p,investment_total_log1p
0,44,2011.0,0.000000,0.000000,0.000000,0,0,0,0.000000,0.000000,0.000000
1,44,2013.0,1803.000000,18606.000000,3101.000000,2,2,2,7.497762,9.831293,8.039803
2,44,2014.0,49.980000,109271.437500,0.000000,2,2,0,3.931433,11.601600,0.000000
3,44,2016.0,4.800000,1441.109985,1310.099976,1,1,1,1.757858,7.273862,7.178622
4,44,2017.0,602.700012,1062.063965,3188.782471,2,1,1,6.403078,6.968911,8.067708


In [94]:
data.drop(columns=['co2_total', 'msaving_total','investment_total'], inplace=True)

In [95]:
# Handling NA values and creating indicator columns for missingness
for column in ['co2_total_log1p', 'msaving_total_log1p', 'investment_total_log1p']:
    # Creating indicator column for missingness
    data[f'{column}_missing'] = data[column].isna().astype(int)

    # Replacing NA with zero in the original columns
    data[column].fillna(0, inplace=True)

# Showing the modified dataframe with the new columns
data.head()

,id,year,co2_counter,msaving_counter,investment_counter,co2_total_log1p,msaving_total_log1p,investment_total_log1p,co2_total_log1p_missing,msaving_total_log1p_missing,investment_total_log1p_missing
0,44,2011.0,0,0,0,0.000000,0.000000,0.000000,0,0,0
1,44,2013.0,2,2,2,7.497762,9.831293,8.039803,0,0,0
2,44,2014.0,2,2,0,3.931433,11.601600,0.000000,0,0,0
3,44,2016.0,1,1,1,1.757858,7.273862,7.178622,0,0,0
4,44,2017.0,2,1,1,6.403078,6.968911,8.067708,0,0,0


In [96]:
data.shape

(25087, 11)

In [97]:
data.to_csv('../../data/processed/cdp_initiative_firm_year_processed.csv', index=False)